In [ ]:
!pip3 install -r requirements.txt

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from matplotlib import pyplot as plt

import mlxtend
from scikeras.wrappers import KerasRegressor
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

In [2]:
from tensorflow.keras import backend as K


def pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.sum(xm * ym)
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = K.sqrt(x_square_sum * y_square_sum)
    r = r_num / r_den
    return K.mean(r)

In [ ]:
vcrdci_all_data = pd.read_csv(
    "vcrdci_123_all_data_eg_no_nans.csv",
    skiprows=1,
    names=[
        "media",
        "mos",
        "raw_mos",
        "EgCodecCategory",
        "S-PanSpeed",
        "S-Jiggle",
        "S-FineDetail",
        "S-WhiteLevel",
        "S-BlackLevel",
        "WhiteClipping",
        "S-Blur",
        "viqet-sharpness",
        "NR-IQA-CDI mean",
        "NR-IQA-CDI std",
        "NR-IQA-CDI entropy",
        "NR-IQA-CDI kurtosis",
        "NR-IQA-CDI skewness",
        "bps",
        "eps",
        "mean_error",
        "Ifrac",
        "Pfrac",
        "Bfrac",
        "mean_countP_countI_ratio",
        "bps_pixels",
        "max_relational_error",
        "mean_relational_error",
        "std_raw_vs_all_error",
        "mean_raw_vs_all_error",
        "max_relational_bits",
        "mean_relational_bits",
        "std_raw_vs_all_bits",
        "mean_raw_vs_all_bits",
        "max_relational_countP",
        "mean_relational_countP",
        "std_raw_vs_all_countP",
        "mean_raw_vs_all_countP",
        "mean_error_vs_mean_bits",
        "mean_countP_vs_mean_bits",
        "max_bps",
        "max_to_mean_bits_per_frame",
    ],
)
vcrdci_all_data.pop("media")
vcrdci_all_data.pop("mos")

In [5]:
vcrdci_60 = vcrdci_all_data[vcrdci_all_data["raw_mos"] >= 60]

In [6]:
vcrdci_60_small = vcrdci_60.iloc[:500]

In [7]:
vcrdci_all_features = vcrdci_60_small.copy()
vcrdci_all_labels = vcrdci_all_features.pop("raw_mos")

In [8]:
categorical_features = ["EgCodecCategory"]
one_hot = OneHotEncoder()
transformer = ColumnTransformer([("one_hot", one_hot, categorical_features)], remainder = "passthrough")

transformed_features = transformer.fit_transform(vcrdci_all_features)
vcrdci_all_features = pd.DataFrame(transformed_features)

In [9]:
feature_train, feature_test, label_train, label_test = train_test_split(vcrdci_all_features, vcrdci_all_labels, test_size = 0.2, random_state = 1)

In [10]:
feature_train = np.array(feature_train)
label_train = np.array(label_train)
feature_test = np.array(feature_test)
label_test = np.array(label_test)

In [11]:
def build_model_sfs(train_input):
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(
                500,
                activation="relu",
                input_shape=((train_input.shape[1]),),
                kernel_regularizer=tf.keras.regularizers.l2(0.001),
            ),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(
                500,
                activation="relu",
                kernel_regularizer=tf.keras.regularizers.l2(0.001),
            ),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1),
        ]
    )

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=["mse", pearson_r],
    )
    return model

In [12]:
tf.keras.backend.clear_session()

class PrintDotSFS(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:
            print("")
        print(".", end="")

early_stop_sfs = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

In [13]:
class MakeModel(object):
    def __init__(self, X=None, y=None):
        pass

    def predict(self, X):
        y_pred = self.model.predict(X)
        return y_pred

    def fit(self, X, y):
        skwrapped_model = KerasRegressor(
            model=build_model_sfs,
            train_input=X,
            epochs=1000,
            batch_size=64,
            validation_split=0.2,
            verbose=0,
            callbacks=[early_stop_sfs, PrintDotSFS()],
        )
        self.model = skwrapped_model
        self.model.fit(X, y)
        return self.model


sffs = SFS(
    MakeModel(),
    k_features=(1, feature_train.shape[1]),
    forward=True,
    floating=False,
    clone_estimator=False,
    cv=0,
    n_jobs=1,
    scoring="neg_mean_squared_error",
    verbose=0,
)

In [14]:
norm_features = tf.keras.layers.Normalization()
norm_features.adapt(vcrdci_all_features)
norm_train = pd.DataFrame(norm_features(feature_train))

In [ ]:
sffs = sffs.fit(norm_train, label_train)

In [ ]:
print("Best accuracy score: %.2f" % sffs.k_score_)
print("Best subset (indices):", sffs.k_feature_idx_)

fig1 = plot_sfs(sffs.get_metric_dict())
lowest_val = pd.DataFrame.from_dict(sffs.get_metric_dict())["avg_score"].min()
plt.ylim([lowest_val, 0])
plt.title("Sequential Forward Selection")
plt.show()

In [ ]:
feature_train_sfs = sffs.transform(feature_train)
feature_train_sfs = np.array(feature_train_sfs)
feature_test_sfs = sffs.transform(feature_test)
feature_test.to_csv("feature_test_vcrdci123_sfs_small.csv", index=False)
feature_test_sfs = np.array(feature_test_sfs)

norm_features_sfs = tf.keras.layers.Normalization()
norm_features_sfs.adapt(sffs.transform(vcrdci_all_features))

In [ ]:
def build_model():
    model = tf.keras.Sequential(
        [
            norm_features_sfs,
            tf.keras.layers.Dense(
                500,
                activation="relu",
                input_shape=[len(feature_train[0])],
                kernel_regularizer=tf.keras.regularizers.l2(0.001),
            ),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(
                500,
                activation="relu",
                kernel_regularizer=tf.keras.regularizers.l2(0.001),
            ),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1),
        ]
    )

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=["mse", pearson_r],
    )
    return model


model = build_model()


class PrintDot(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:
            print("")
        print(".", end="")


early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=100, restore_best_weights=True
)

history = model.fit(
    feature_train_sfs,
    label_train,
    epochs=1000,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop, PrintDot()],
    verbose=0,
)

hist = pd.DataFrame(history.history)
hist["epoch"] = history.epoch

val_rmse = np.sqrt(float(hist["val_mse"].min()))
val_pearson = float(hist["val_pearson_r"].min())
print("\nFinal Root Mean Square Error on validation set: {}".format(round(val_rmse, 3)))
print(
    "Final Pearson Correlation Error on validation set: {}".format(
        round(val_pearson, 3)
    )
)

print("\nEvaluating...")
scores = model.evaluate(feature_test_sfs, label_test, verbose=0)
test_loss_mse, test_metric_mse, test_metric_pearson = scores

test_loss_rmse = np.sqrt(test_loss_mse)

print("Loss: RMSE on test set: {}".format(round(test_loss_rmse, 3)))
print("Metric: Pearson on test set: {}".format(round(test_metric_pearson, 3)))


def plot_history():
    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Root Mean Square Error [VMAF Score]")
    plt.plot(hist["epoch"], np.sqrt(hist["mse"]), label="Train Error")
    plt.plot(hist["epoch"], np.sqrt(hist["val_mse"]), label="Val Error")
    plt.legend()
    plt.ylim([0, 100])
    plt.show()


plot_history()

In [ ]:
model.save("/content/vcrdci123_small_model_test", save_format="tf")
!zip -r vcrdci123_small_model_test.zip vcrdci123_small_model_test/

In [ ]:
feature_test = pd.read_csv("feature_test_vcrdci123_small.csv")
feature_test = np.array(feature_test)

In [ ]:
predictions = model.predict(feature_test)

In [ ]:
with open("tf_predictions.txt", "w+") as f:
  f.write(str(predictions))